# テキスト分析

自然言語処理（NLP）は、人工知能（AI）の一分野で、書き言葉や話し言葉を扱います。NLPを利用して、テキストや音声から意味を抽出したり、自然言語で意味のある応答を形成したりするソリューションを構築することができます。

Microsoft Azureの**Cognitive Services**には、**Text Analytics**サービスが含まれており、テキスト内のキーフレーズの識別や、センチメントに基づいたテキストの分類など、すぐに使えるNLP機能が提供されています。

![A robot reading a notebook](./images/NLP.jpg)

例えば、架空の組織*Margie's Travel*では、顧客がホテルに宿泊した際にレビューを投稿することを奨励しているとします。テキスト分析サービスを使って、キーフレーズを抽出してレビューを要約したり、どのレビューが肯定的でどのレビューが否定的かを判断したり、場所や人などの既知のエンティティに言及しているレビューテキストを分析したりすることができます。

## レビューのテキストを見る

まずは、お客様から寄せられたホテルのレビューを見てみましょう。

レビューはテキストファイルになっています。以下のコードを実行するには、セルの左にある**Run cell** (&#9655;)ボタンをクリックしてください。

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Cognitive Services リソースを作成する

これらのレビューのテキストを分析するには、**Text Analytics** Cognitive Serviceを使用することができます。これを使用するには、Azureサブスクリプションで**Text Analytics**または**Cognitive Services**リソースのいずれかをプロビジョニングする必要があります(このサービスのみを使用する予定の場合や、使用状況を個別に追跡したい場合はText Analyticsリソースを使用し、そうでない場合はCognitive Servicesリソースを使用してText Analyticsサービスと他のコグニティブサービスを組み合わせ、開発者が単一のエンドポイントとキーを使用してアクセスできるようにします)。

まだ作成していない方は、以下の手順でAzureサブスクリプションに**Cognitive Services**リソースを作成してください。

> **補足** すでにCognitive Servicesリソースがある場合は、Azureポータルでその**クイックスタート**ページを開き、そのキーとエンドポイントを下のセルにコピーするだけです。そうでない場合は、以下の手順で作成してください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。
2. **&#65291;リソースの作成** ボタンをクリックし、*Cognitive Services*を検索して以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *ご自身のサプスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: S0
    - **このボックスをオンにすることにより、以下のすべてのご契約条件を読み、同意したものとみなされます**: チェックを入れます。
3. デプロイが完了するまでしばらく待ちます。次に、Cognitive Servicesリソースにアクセスし、**Overview**ページで、サービスのキーを管理するリンクをクリックします。クライアントアプリケーションからCognitive Servicesリソースに接続するには、エンドポイントとキーが必要になります。


### Cognitive Servicesリソースのキーとエンドポイントの取得

Cognitive Servicesリソースを使用するためには、クライアントアプリケーションはそのエンドポイントと認証キーを必要とします。

1. Azureポータルで、Cognitive Servicesリソースの**キーとエンドポイント**ページを開き、リソースの**キー1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。

2. リソースの**エンドポイント**をコピーし、**YOUR_COG_ENDPOINT**を置き換えて、以下のコードに貼り付けます。

3. セルの実行<span>&#9655;</span>ボタン（セルの左上）をクリックして、下のセルのコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'
print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## 言語検出
はじめに、これらのレビューが書かれている言語を検出することから始めましょう。

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## キーフレーズ抽出

次に、カスタマーレビューの文章を分析して、主な話題のヒントとなるキーフレーズを見つけます。

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

キーフレーズは、各レビューの最も重要な論点を理解するのに役立ちます。例えば、「helpful staff」や「poor service」というフレーズが含まれているレビューを見ると、そのレビューを書いた人の主な関心事がわかります。

## 感情分析

**センチメント・スコア**に基づいて、レビューを**ポジティブ**または**ネガティブ**に分類すると便利かもしれません。この場合も、テキスト分析サービスを利用することができます。

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## エンティティの抽出

**エンティティ**とは、一般的に理解されているタイプのアイテムを参照するテキストで言及される可能性のあるものです。例えば、場所、人、日付などです。例えば、レビューに記載されている日付や場所に興味があるとします。以下のコードを使用してそれらを見つけることができます。

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

なお、エンティティの中にはWikipediaにページがあるほど有名なものもあります。その場合はText AnalyticsサービスがそのページのURLを返します。

## さらに学ぶ

Text Analyticsサービスの詳細については、[Azure Cognitive Service for Language のドキュメント](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)を参照してください。